In [9]:
from copy import deepcopy
import os
from pprint import pprint
import sys

import yaml


path_petsard = os.path.dirname(os.path.dirname(os.getcwd()))
print(path_petsard)
sys.path.append(path_petsard)


filename = 'Exec_Yaml.yaml'

with open(filename, 'r') as yaml_file:
    yaml_raw: dict = yaml.safe_load(yaml_file)

pprint (yaml_raw)

{'Evaluator': {'sd-qlt': {'method': 'sdmetrics-single_table-qualityreport'}},
 'Loader': {'adult': {'filepath': 'benchmark://adult',
                      'na_values': {'native-country': '?',
                                    'occupation': '?',
                                    'workclass': '?'}}},
 'Postprocessor': {'missing-drop': None},
 'Preprocessor': {'missing-drop': {'encoder': {'education': 'encoder_label',
                                               'gender': 'encoder_label',
                                               'income': 'encoder_uniform',
                                               'marital-status': 'encoder_label',
                                               'native-country': 'encoder_uniform',
                                               'occupation': 'encoder_uniform',
                                               'race': 'encoder_uniform',
                                               'relationship': 'encoder_label',
                           

In [10]:
# _splitter_handler

from PETsARD.config import Config

cfg = Config(filename=filename)

pprint (yaml_raw['Splitter'])
pprint (cfg.yaml['Splitter'])

{'0.5': {'num_samples': 2, 'train_split_ratio': 0.5},
 '0.8': {'num_samples': 2, 'train_split_ratio': 0.8}}
{'0.5_[1|2]': {'num_samples': 1, 'train_split_ratio': 0.5},
 '0.5_[2|2]': {'num_samples': 1, 'train_split_ratio': 0.5},
 '0.8_[1|2]': {'num_samples': 1, 'train_split_ratio': 0.8},
 '0.8_[2|2]': {'num_samples': 1, 'train_split_ratio': 0.8}}


In [11]:
from PETsARD.operator import LoaderOperator

loading = LoaderOperator(config=cfg.yaml['Loader']['adult'])
loading.run(input={})
pprint(loading.get_result().head(2))

Loader - Benchmarker: file benchmark\adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
   age workclass  fnlwgt education  educational-num      marital-status  \
0   25   Private  226802      11th                7       Never-married   
1   38   Private   89814   HS-grad                9  Married-civ-spouse   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   
1    Farming-fishing      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
1              50  United-States  <=50K  


In [12]:
from PETsARD.operator import SplitterOperator

splitting = SplitterOperator(config=cfg.yaml['Splitter']['0.8_[1|2]'])
splitting.run(input={'data': loading.get_result()})
print('train')
pprint(splitting.get_result()['train'].head(2))
print('validation')
pprint(splitting.get_result()['validation'].head(2))


train
   age workclass  fnlwgt education  educational-num      marital-status  \
0   25   Private  226802      11th                7       Never-married   
1   38   Private   89814   HS-grad                9  Married-civ-spouse   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   
1    Farming-fishing      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
1              50  United-States  <=50K  
validation
   age workclass  fnlwgt     education  educational-num      marital-status  \
0   27   Private  205145       HS-grad                9  Married-civ-spouse   
1   22   Private  401451  Some-college               10  Married-civ-spouse   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct      Husband  White   Male             0             0 

In [13]:
import warnings

from PETsARD.operator import PreprocessorOperator

with warnings.catch_warnings():
    # 忽略 FutureWarning. See Issue # 244: https://github.com/matheme-justyn/PETsARD/issues/244
    warnings.simplefilter(action='ignore', category=FutureWarning)

    preproc = PreprocessorOperator(config=cfg.yaml['Preprocessor']['missing-drop'])
    preproc.run(input={
        'data': splitting.get_result()['train'],
        'metadata': loading.loader.metadata
    })
    pprint(preproc.get_result().head(2))

        age  workclass    fnlwgt  education  educational-num  marital-status  \
0 -0.996639   0.546648  0.349771          1        -1.193814               4   
1 -0.049669   0.600708 -0.941413         11        -0.415929               2   

   occupation  relationship      race  gender  capital-gain  capital-loss  \
0    0.796283             3  0.913585       1      -0.14482     -0.218644   
1    0.919238             0  0.143268       1      -0.14482     -0.218644   

   hours-per-week  native-country    income  
0       -0.034804        0.803456  0.193865  
1        0.770678        0.151252  0.559039  


In [14]:
from PETsARD.operator import SynthesizerOperator

syn = SynthesizerOperator(config=cfg.yaml['Synthesizer']['sdv-gaussian'])
syn.run(input={'data': preproc.get_result()})
pprint(syn.get_result().head(2))


Synthesizer (SDV - SingleTable): Metafile loading time: 0.0507 sec.
Synthesizer (SDV - SingleTable): Fitting GaussianCopula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCopula spent 8.0091 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCopula # 21489 rows (same as raw) in 1.3653 sec.
        age  workclass    fnlwgt  education  educational-num  marital-status  \
0  1.232736   0.583504 -0.221518          7         1.558109               2   
1 -0.790078   0.517925  0.273535         11         0.929941               2   

   occupation  relationship      race  gender  capital-gain  capital-loss  \
0    0.558875             1  0.399017       0      -0.14482     -0.218644   
1    0.321746             0  0.785830       1      -0.14482     -0.218644   

   hours-per-week  native-country    income  
0        0.155380        0.590887  0.399915  
1        0.641833        0.431233  0.019265  


In [15]:
from PETsARD.operator import PostprocessorOperator

postproc = PostprocessorOperator(config=cfg.yaml['Postprocessor']['missing-drop'])
postproc.run(input={
    'preprocessor': preproc.processor,
    'data': syn.get_result()
})
pprint(postproc.get_result().head(2))

         age workclass         fnlwgt   education  educational-num  \
0  55.604854   Private  166191.195050  Assoc-acdm        14.075397   
1  27.835664   Private  218713.780869     HS-grad        12.460329   

       marital-status occupation   relationship   race  gender  capital-gain  \
0  Married-civ-spouse      Sales  Not-in-family  White  Female           0.0   
1  Married-civ-spouse        NaN        Husband  White    Male           0.0   

   capital-loss  hours-per-week native-country income  
0           0.0       42.361124  United-States  <=50K  
1           0.0       48.400394  United-States  <=50K  


In [16]:
from PETsARD.operator import EvaluatorOperator

evaluating = EvaluatorOperator(config=cfg.yaml['Evaluator']['sd-qlt'])
evaluating.run(input={
    'data': {
        'ori': splitting.get_result()['train'],
        'syn': postproc.get_result(),
        'control': splitting.get_result()['validation'],
    }
})
pprint(evaluating.get_result()['details']['Column Pair Trends'].head(2))

Evaluator (SDMetrics): Evaluating QualityReport.
Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 105/105 [00:04<00:00, 22.45it/s]

Overall Score: 74.18%

Properties:
- Column Shapes: 91.0%
- Column Pair Trends: 57.37%
Evaluator (SDMetrics): Evaluating QualityReport spent 4.9107 sec.
  Column 1   Column 2                 Metric     Score  Real Correlation  \
0      age  workclass  ContingencySimilarity  0.865451               NaN   
1      age     fnlwgt  CorrelationSimilarity  0.997742         -0.076085   

   Synthetic Correlation Error  
0                    NaN  None  
1               -0.07157  None  


In [1]:
from copy import deepcopy
import sys
from pprint import pprint


sys.path.append('D:/Dropbox/89_other_application/GitHub/PETsARD')
filename = 'DevTest_Exec_Yaml.yaml'

from PETsARD.config import Config, Status

cfg = Config(filename=filename)

# pprint([cfg.config.get()      for _ in range(cfg.config.qsize()     )])
pprint([cfg.module_flow.get() for _ in range(cfg.module_flow.qsize())])
pprint([cfg.expt_flow.get()   for _ in range(cfg.expt_flow.qsize()  )])

['Loader',
 'Splitter',
 'Preprocessor',
 'Synthesizer',
 'Postprocessor',
 'Evaluator',
 'Splitter',
 'Preprocessor',
 'Synthesizer',
 'Postprocessor',
 'Evaluator',
 'Splitter',
 'Preprocessor',
 'Synthesizer',
 'Postprocessor',
 'Evaluator',
 'Splitter',
 'Preprocessor',
 'Synthesizer',
 'Postprocessor',
 'Evaluator']
['adult',
 '0.8_[1|2]',
 'missing-drop',
 'sdv-gaussian',
 'missing-drop',
 'sd-qlt',
 '0.8_[2|2]',
 'missing-drop',
 'sdv-gaussian',
 'missing-drop',
 'sd-qlt',
 '0.5_[1|2]',
 'missing-drop',
 'sdv-gaussian',
 'missing-drop',
 'sd-qlt',
 '0.5_[2|2]',
 'missing-drop',
 'sdv-gaussian',
 'missing-drop',
 'sd-qlt']


In [3]:
cfg = Config(filename=filename)
sts: Status = Status(config=cfg)
pre_module: str = ''
result = {}

while cfg.config.qsize() > 0:
    ops    = cfg.config.get()
    module = cfg.module_flow.get()
    expt   = cfg.expt_flow.get()
    exclude_index: list = []

    print(f"module {module}")

    ops.run(ops.set_input(status=sts))

    sts.put(module, expt, ops)

    if module == cfg.sequence[-1]:
        full_expt = sts.get_full_expt()
        full_expt_str = '_'.join(
            [f"{module}[{expt}]" for module, expt in full_expt.items()]
        )
        result[full_expt_str] = deepcopy(sts.get_result(module=module))


module Loader
Loader - Benchmarker: file benchmark\adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
module Splitter
module Preprocessor
module Synthesizer
Synthesizer (SDV - SingleTable): Metafile loading time: 0.0786 sec.
Synthesizer (SDV - SingleTable): Fitting GaussianCopula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCopula spent 10.2012 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCopula # 21661 rows (same as raw) in 1.4333 sec.
module Postprocessor
module Evaluator
Evaluator (SDMetrics): Evaluating QualityReport.
Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 105/105 [00:04<00:00, 24.42it/s]

Overall Score: 73.44%

Properties:
- Column Shapes: 90.43%
- Column Pair Trends: 56.45%
Evaluator (SDMetrics): Evaluating QualityReport spent 4.5076 sec.
module Splitter
module Preprocessor
module Synthesizer
Synthesizer (SDV - SingleTable): Metafile loading time: 0.0156 

In [7]:
import pprint
pp = pprint.PrettyPrinter(depth=1)
pp.pprint(result)

{'Loader[adult]_Splitter[0.5_[1|2]]_Preprocessor[missing-drop]_Synthesizer[sdv-gaussian]_Postprocessor[missing-drop]_Evaluator[sd-qlt]': {'details': {'Column Pair Trends':            Column 1         Column 2                 Metric     Score  \
0               age        workclass  ContingencySimilarity  0.864082   
1               age           fnlwgt  CorrelationSimilarity  0.997330   
2               age        education  ContingencySimilarity  0.387922   
3               age  educational-num  CorrelationSimilarity  0.994071   
4               age   marital-status  ContingencySimilarity  0.752679   
..              ...              ...                    ...       ...   
100    capital-loss   native-country  ContingencySimilarity  0.006675   
101    capital-loss           income  ContingencySimilarity  0.006675   
102  hours-per-week   native-country  ContingencySimilarity  0.523088   
103  hours-per-week           income  ContingencySimilarity  0.512204   
104  native-country      

In [10]:
from copy import deepcopy
import sys
from pprint import pprint


sys.path.append('D:/Dropbox/89_other_application/GitHub/PETsARD')
filename = 'DevTest_Exec_Yaml.yaml'

from PETsARD import Executor

exec = Executor(config=filename)
exec.run()


Now is Loader with adult...
Loader - Benchmarker: file benchmark\adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
Now is Splitter with 0.8_[1|2]...
Now is Preprocessor with missing-drop...
Now is Synthesizer with sdv-gaussian...
Synthesizer (SDV - SingleTable): Metafile loading time: 0.0156 sec.
Synthesizer (SDV - SingleTable): Fitting GaussianCopula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCopula spent 8.7702 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCopula # 21595 rows (same as raw) in 1.7304 sec.
Now is Postprocessor with missing-drop...
Now is Evaluator with sd-qlt...
Evaluator (SDMetrics): Evaluating QualityReport.
Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 105/105 [00:05<00:00, 18.26it/s]

Overall Score: 73.91%

Properties:
- Column Shapes: 90.85%
- Column Pair Trends: 56.98%
Evaluator (SDMetrics): Evaluating QualityReport spent 5.9479 sec.
Now is Sp

ValueError: bins must increase monotonically.

In [11]:
exec.get_result()

{'Loader[adult]_Splitter[0.8_[1|2]]_Preprocessor[missing-drop]_Synthesizer[sdv-gaussian]_Postprocessor[missing-drop]_Evaluator[sd-qlt]': {'score': 0.7391440757657324,
  'properties': {'Column Shapes': {'Score': 0.9085121891000267},
   'Column Pair Trends': {'Score': 0.5697759624314381}},
  'details': {'Column Shapes':              Column        Metric     Score
   0               age  KSComplement  0.939809
   1         workclass  TVComplement  0.993492
   2            fnlwgt  KSComplement  0.955519
   3         education  TVComplement  0.504252
   4   educational-num  KSComplement  0.794910
   5    marital-status  TVComplement  0.967206
   6        occupation  TVComplement  0.978661
   7      relationship  TVComplement  0.963878
   8              race  TVComplement  0.994605
   9            gender  TVComplement  0.997304
   10     capital-gain  KSComplement  0.917974
   11     capital-loss  KSComplement  0.953523
   12   hours-per-week  KSComplement  0.692639
   13   native-country  T

In [1]:
from copy import deepcopy
import os
from pprint import pprint
import sys

import yaml


path_petsard = os.path.dirname(os.path.dirname(os.getcwd()))
print(path_petsard)
sys.path.append(path_petsard)


filename = 'Issue290.yaml'

with open(filename, 'r') as yaml_file:
    yaml_raw: dict = yaml.safe_load(yaml_file)

pprint (yaml_raw)

from PETsARD import Config

cfg = Config(filename=filename)

from PETsARD.operator import LoaderOperator

loading = LoaderOperator(config=cfg.yaml['Loader']['adult'])
loading.run(input={})
pprint(loading.get_result().head(2))

d:\Dropbox\89_other_application\GitHub\PETsARD
{'Loader': {'adult': {'filepath': 'benchmark://adult',
                      'na_values': {'native-country': '?',
                                    'occupation': '?',
                                    'workclass': '?'}}},
 'Preprocessor': {'normal': {'method': 'default',
                             'sequence': ['missing',
                                          'outlier',
                                          'encoder',
                                          'scaler']},
                  'smartnoise': {'method': 'default',
                                 'sequence': ['missing',
                                              'outlier',
                                              'scaler',
                                              'discretizing']}}}
Loader - Benchmarker: file benchmark\adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
   age workclas

In [2]:
from PETsARD.operator import PreprocessorOperator

preproc = PreprocessorOperator(config=cfg.yaml['Preprocessor']['normal'])
preproc.run(input={
    'data': loading.get_result(),
    'metadata': loading.loader.metadata
})
pprint(preproc.get_result().head(2))

        age  workclass    fnlwgt  education  educational-num  marital-status  \
0 -0.995129   0.053867  0.351675   0.816707        -1.197259        0.488336   
1 -0.046942   0.219780 -0.945524   0.281669        -0.419335        0.439379   

   occupation  relationship      race    gender  capital-gain  capital-loss  \
0    0.751032      0.675467  0.877865  0.375161     -0.144804     -0.217127   
1    0.933054      0.264872  0.110144  0.138663     -0.144804     -0.217127   

   hours-per-week  native-country    income  
0       -0.034087        0.610481  0.646800  
1        0.772930        0.370624  0.723263  


In [3]:
preproc = PreprocessorOperator(config=cfg.yaml['Preprocessor']['smartnoise'])
preproc.run(input={
    'data': loading.get_result(),
    'metadata': loading.loader.metadata
})
pprint(preproc.get_result().head(2))

        age  workclass    fnlwgt  education  educational-num  marital-status  \
0 -0.995129   0.601461  0.351675   0.843180        -1.197259        0.630587   
1 -0.046942   0.193471 -0.945524   0.087336        -0.419335        0.288118   

   occupation  relationship      race    gender  capital-gain  capital-loss  \
0    0.748999      0.799254  0.945539  0.041649     -0.144804     -0.217127   
1    0.938767      0.016340  0.591570  0.501739     -0.144804     -0.217127   

   hours-per-week  native-country    income  
0       -0.034087        0.506326  0.570303  
1        0.772930        0.529629  0.348880  
